In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
# sys.path.append('/home/pedrorozin/scripts/python') #uncomment para agregar el path. yo lo tengo instalado en mi venv
import funciones_tesis as ft


## Comparación $\delta_m$ vs $\delta_{cdm}$ para distintos k's

In [2]:
path = '/home/pedrorozin/scripts/class_public/delta_prime_cdm.txt'
ini_path = '/home/pedrorozin/scripts/class_public/test_delta_prime_gauge_newtonian.ini'
ft.read_ini_params(ini_path)


df = ft.read_adhoc_txt(path)
df['delta_m'] = ft.compute_delta_m(df)
df.head()

TypeError: compute_delta_m() missing 3 required positional arguments: 'delta_b', 'omega_cdm', and 'omega_b'

In [5]:
# Solution 1: Reload the module to refresh docstrings
import importlib
import funciones_tesis as ft
importlib.reload(ft)

# Solution 2: Check if docstrings are available
print("Docstring for compute_delta_m:")
print(ft.compute_delta_m.__doc__)
print("\n" + "="*50 + "\n")

# Solution 3: Check help information
print("Help for compute_delta_m:")
help(ft.compute_delta_m)

Docstring for compute_delta_m:

  Compute the total matter density perturbation from CDM and baryonic perturbations.

  Args:
    delta_cdm (float): CDM density perturbation.
    delta_b (float): Baryonic density perturbation.
    omega_cdm (float): Omega_cdm value.
    omega_b (float): Omega_b value.

  Returns:
    array or float128: Total matter density perturbation.
  


Help for compute_delta_m:
Help on function compute_delta_m in module funciones_tesis:

compute_delta_m(delta_cdm, delta_b, omega_cdm, omega_b)
    Compute the total matter density perturbation from CDM and baryonic perturbations.
    
    Args:
      delta_cdm (float): CDM density perturbation.
      delta_b (float): Baryonic density perturbation.
      omega_cdm (float): Omega_cdm value.
      omega_b (float): Omega_b value.
    
    Returns:
      array or float128: Total matter density perturbation.



## Soluciones para ver docstrings en VS Code

El problema es que VS Code a veces no reconoce los docstrings de módulos instalados. Aquí están las soluciones:

### ✅ Solución 1: Usar `importlib.reload()`
Ya ejecutamos esto arriba - recarga el módulo para que VS Code reconozca los docstrings.

### ✅ Solución 2: Reiniciar el Kernel
- Ve al menú: `Kernel` → `Restart`
- Luego vuelve a ejecutar tus celdas de import

### ✅ Solución 3: Usar extensiones de Python
- Asegúrate de tener instalada la extensión oficial de Python de Microsoft
- Instala también la extensión "Pylance" para mejor soporte de IntelliSense

### ✅ Solución 4: Configurar VS Code settings
Agrega esto a tu `settings.json` de VS Code:
```json
{
    "python.analysis.autoImportCompletions": true,
    "python.analysis.typeCheckingMode": "basic"
}
```

### ✅ Solución 5: Si nada funciona
Usa `help(ft.función_name)` o `ft.función_name.__doc__` directamente en el código.

In [6]:
# Test: Hover over these function names to see if docstrings appear
# After running the reload above, these should show documentation on hover:

# Hover over 'read_adhoc_txt' - should show its docstring
data = ft.read_adhoc_txt

# Hover over 'compute_delta_m' - should show its docstring  
compute_func = ft.compute_delta_m

# Hover over 'common_settings' - should show its docstring
settings_func = ft.common_settings

print("✅ Try hovering over the function names above!")
print("If it still doesn't work, try:")
print("1. Restart the kernel (Kernel → Restart)")
print("2. Re-run the import cells")
print("3. Check if you have the Python and Pylance extensions installed")

✅ Try hovering over the function names above!
If it still doesn't work, try:
1. Restart the kernel (Kernel → Restart)
2. Re-run the import cells
3. Check if you have the Python and Pylance extensions installed
